<a href="https://colab.research.google.com/github/HP-Nunes/SMCapstone_GColab/blob/main/Notebook_2_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. Transformation**


### Boilerplate Code

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -r drive/MyDrive/Gcolab/requirements.txt

Mounted at /content/drive
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 23.9 MB 1.1 MB/s 
     |████████████████████████████████| 1.0 MB 60.8 MB/s 
     |████████████████████████████████| 1.8 MB 72.2 MB/s 
     |████████████████████████████████| 6.3 MB 61.6 MB/s 
     |████████████████████████████████| 15.4 MB 82.7 MB/s 
     |████████████████████████████████| 19.3 MB 54.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import geopandas as gpd
import contextily # https://geographicdata.science/book/notebooks/08_point_pattern_analysis.html
import pygeos
import shapely.speedups
from pyproj import CRS
from fiona.crs import to_string, from_epsg
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Data Cleaning
---
### Objectives

*   Standardize the data schema across all years;
*   Filter the records across all sets (i.e. no datapoints outside of the city of San Francisco, remove NULLs and 'artificial' records [errors, strange outliers, testing sites]);
*   Identify the attributes of interests (and drop redundant/disinterested fields), and convert the fields to the appropriate data-type if need be;




In [2]:
# Import the raw csv yearly datasets into their respective pandas dataframe

df19raw = pd.read_csv(r'drive/MyDrive/Gcolab/rawdata/systemdatalyft/2019lyftraw.csv',float_precision=None)
display(df19raw.shape)
df20raw = pd.read_csv(r'drive/MyDrive/Gcolab/rawdata/systemdatalyft/2020lyftraw.csv',float_precision=None)
display(df20raw.shape)
df21raw = pd.read_csv(r'drive/MyDrive/Gcolab/rawdata/systemdatalyft/2021lyftraw.csv',float_precision=None)
display(df21raw.shape)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2506983, 16)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,5,7,12,14,16,17,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2143557, 25)

(1492990, 14)

In [3]:
def cleaning2019(df):

  columns = ['start_time','end_time']
  for column in columns:
    df[column] = pd.to_datetime(df[column],dayfirst=True)       
    df[column + str('_day')] = df[column].dt.day_name()
    df[column + str('_month')] = df[column].dt.month_name()
    df[column + str('_hour')] = df[column].dt.round('H').dt.hour
    df[column + str('_year')] = df[column].dt.year

  df = df.drop(['fname'],axis=1) # not needed

  df = df[df.start_station_name != 'Prototype Lab'] # removing test trips 

  df = df[(df['start_station_latitude'] < 37.82) & (df['start_station_latitude'] > 37.7) & (df['start_station_longitude'] < -122.36) & (df['start_station_longitude'] > -122.52)]
      # retain locations only for the city of San Francisco
      
  cols = ['start_station_latitude', 'start_station_longitude','end_station_latitude','end_station_longitude']
  df[cols] = df[cols].round(6)
      
  df['duration_min'] = df['duration_sec'] // 60

  df['bike_share_for_all_trip'] = df['bike_share_for_all_trip'].fillna('No')

  # Create a categorical variable to distinguish between Docked-to-Docked, Dockless-to-Docked, Docked-to-Dockless, and Dockless-to-Dockless trips
  df['trip_type'] = ''
  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].notnull()),'trip_type'] = 'Docked-to-Docked'
  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].isnull()),'trip_type'] = 'Docked-to-Dockless'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].notnull()),'trip_type'] = 'Dockless-to-Docked'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].isnull()),'trip_type'] = 'Dockless-to-Dockless'

  df = df.sort_values(by='start_time')

  ## ! Important: the formal ebike_trip designation has been changed to meet the 2020 dataset's schema
  ## It is now referred to as rideable_type with outputs either as classic_bike or electric_bike.
  df['rideable_type'] = ""
  df.loc[df.trip_type == 'Docked-to-Docked', 'rideable_type'] = "classic_bike" # A "No" means "Classic Bike";
  ## I am also assuming that all Docked trips occur via Classic Bikes, although they could also include eBikes
  df.loc[df.trip_type != 'Docked-to-Docked', 'rideable_type'] = "electric_bike"
  df['est_cost'] = ""
  df['est_cost'] = pd.to_numeric(df['est_cost'], downcast='integer')

  df = df.drop_duplicates() # drops duplicated rows (precautionary wrangling)

  return df
##########################################################################################################################################
##########################################################################################################################################
def cleaning2020(df):

  columns = ['start_time','end_time','started_at','ended_at']
  for column in columns:
    df[column] = pd.to_datetime(df[column],dayfirst=True)       
    df[column + str('_day')] = df[column].dt.day_name()
    df[column + str('_month')] = df[column].dt.month_name()
    df[column + str('_hour')] = df[column].dt.round('H').dt.hour
    df[column + str('_year')] = df[column].dt.year
  df = df.sort_values(by='start_time')

  # This effectively removes the 3,342 rows with no data for end points.
  df = df.loc[~((df['end_station_latitude'].isnull()) & (df['end_station_longitude'].isnull()) & (df['end_lat'].isnull()) & (df['end_lng'].isnull()))]
  # Extracts all coordinates from one set of coordinate fields to another, to keep things standardized.
  df['start_station_longitude'] = df['start_station_longitude'].fillna(df['start_lng'])
  df['start_station_latitude'] = df['start_station_latitude'].fillna(df['start_lat'])
  df['end_station_longitude'] = df['end_station_longitude'].fillna(df['end_lng'])
  df['end_station_latitude'] = df['end_station_latitude'].fillna(df['end_lat'])
  # Ditto with the start time/end time fields.
  df['start_time'] = df['start_time'].fillna(df['started_at'])
  df['end_time'] = df['end_time'].fillna(df['ended_at'])
  # duration_sec has missing values; recalc from reformatted time fields
  df['duration_sec'] = (df['end_time'] - df['start_time']).dt.total_seconds()
  df['duration_min'] = df['duration_sec'] // 60
  df = df[(df['duration_sec'] > 0)] ## native error in the raw dataset
                      ## Some end times have an earlier timestamp than start times.
  ## There are apparent errors in the start and end time fields, where some values seem to be flipped.
      ## This leads to "negative" duration. Not an artifact of wrangling or operations, but of the raw data.
  columns = ['fname','start_lat','start_lng','end_lat','end_lng','started_at','ended_at']
  df = df.drop(columns,axis=1) # these columns are not needed
  df = df[(df['start_station_latitude'] < 37.82) & (df['start_station_latitude'] > 37.7) & (df['start_station_longitude'] < -122.36) & (df['start_station_longitude'] > -122.52)]
      # retain locations only for the city of San Francisco
  cols = ['start_station_latitude', 'start_station_longitude','end_station_latitude','end_station_longitude']
  df[cols] = df[cols].round(6)

  df['trip_type'] = ''

  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].notnull()),'trip_type'] = 'Docked-to-Docked'
  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].isnull()),'trip_type'] = 'Docked-to-Dockless'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].notnull()),'trip_type'] = 'Dockless-to-Docked'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].isnull()),'trip_type'] = 'Dockless-to-Dockless'

  df['est_cost'] = ""
  df['est_cost'] = pd.to_numeric(df['est_cost'], downcast='integer')

  # df['is_equity'] = df['is_equity'].fillna(value=True)

  ## Missing user_type data.
  df.loc[df['member_casual'] == 'casual', 'user_type'] = "Customer" # casual = customer (one time purchase);
  df.loc[df['member_casual'] == 'member', 'user_type'] = "Subscriber" # member = subscriber (membership holder);

  df = df.drop_duplicates()

  return df
##########################################################################################################################################
##########################################################################################################################################
def cleaning2021(df):
  df = df.rename(columns={'started_at': 'start_time','ended_at':'end_time','start_lat':'start_station_latitude',
                     'start_lng':'start_station_longitude','end_lat':'end_station_latitude','end_lng':'end_station_longitude'})
  
  columns = ['start_time','end_time']
  for column in columns:
    df[column] = pd.to_datetime(df[column],dayfirst=True)       
    df[column + str('_day')] = df[column].dt.day_name()
    df[column + str('_month')] = df[column].dt.month_name()
    df[column + str('_hour')] = df[column].dt.round('H').dt.hour
    df[column + str('_year')] = df[column].dt.year

  df = df.sort_values(by='start_time')

  df = df[(df['start_station_latitude'] < 37.82) & (df['start_station_latitude'] > 37.7) & (df['start_station_longitude'] < -122.36) & (df['start_station_longitude'] > -122.52)]
    # retain locations only for the city of San Francisco
    
  cols = ['start_station_latitude', 'start_station_longitude','end_station_latitude','end_station_longitude']
  df[cols] = df[cols].round(6)
      
  df['duration_sec'] = (df['end_time'] - df['start_time']).dt.total_seconds()
  df['duration_min'] = df['duration_sec'] // 60
  df = df[(df['duration_sec'] > 0)]

  df['trip_type'] = ''
  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].notnull()),'trip_type'] = 'Docked-to-Docked'
  df.loc[(df['start_station_id'].notnull() & df['end_station_id'].isnull()),'trip_type'] = 'Docked-to-Dockless'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].notnull()),'trip_type'] = 'Dockless-to-Docked'
  df.loc[(df['start_station_id'].isnull() & df['end_station_id'].isnull()),'trip_type'] = 'Dockless-to-Dockless'

  df.loc[df['member_casual'] == 'casual', 'user_type'] = "Customer" 
  df.loc[df['member_casual'] == 'member', 'user_type'] = "Subscriber"

  df = df.drop_duplicates() # drops duplicated rows (precautionary wrangling)
  
  return df

In [4]:
df19 = cleaning2019(df19raw)
df20 = cleaning2020(df20raw)
df21 = cleaning2021(df21raw)

print("For the 2019 dataset, we have dropped " + str(len(df19raw) - len(df19)) + " rows, about " + str(100 - (100 * (len(df19) / len(df19raw)))) + "% from the original raw dataset.")
print("For the 2020 dataset, we have dropped " + str(len(df20raw) - len(df20)) + " rows, about " + str(100 - (100 * (len(df20) / len(df20raw)))) + "% from the original raw dataset.")
print("For the 2021 dataset, we have dropped " + str(len(df21raw) - len(df21)) + " rows, about " + str(100 - (100 * (len(df21) / len(df21raw)))) + "% from the original raw dataset.")

For the 2019 dataset, we have dropped 654044 rows, about 26.088888516595446% from the original raw dataset.
For the 2020 dataset, we have dropped 389649 rows, about 18.177683168677106% from the original raw dataset.
For the 2021 dataset, we have dropped 217101 rows, about 14.541356606541228% from the original raw dataset.


## Feature Engineering
---
### Objectives

*   Add geospatial feature attribute through spatial intersection with georeferenced datasets;
*   Estimate the revenue generated for each trip;
*   Assign categorical values for each trip.


### Generating New Geospatial Attributes

In [5]:
fname = 'drive/MyDrive/Gcolab/rawdata/DataSF/SF_N.geojson' # Your filepath here
poly = gpd.read_file(fname)
# print(type(poly))
# poly.head()

def pythonicGIS(df):
  # I am intersecting a GeoJSON of San Francisco's neighborhoods (polygon units) for each trip's starting and ending datapoints' location; hence the multiple spatial joins
  df_startN = gpd.GeoDataFrame(
  df, crs='EPSG:4326',
      geometry=gpd.points_from_xy(df.start_station_longitude, df.start_station_latitude))
  type(df_startN) # check here if the conversion worked
  df_startN_int = gpd.sjoin(df_startN,poly,how="inner", predicate='intersects')
  df_startN_int = df_startN_int.rename(columns={'name': 'start_neigh'})

  del df_startN_int['index_right']

  df_endN = gpd.GeoDataFrame(
  df_startN_int, crs='EPSG:4326',
      geometry=gpd.points_from_xy(df_startN_int.end_station_longitude, df_startN_int.end_station_latitude))
  type(df_endN) # check here if the conversion worked
  df_endN_int = gpd.sjoin(df_endN,poly,how="inner", predicate='intersects')
  df_endN_int = df_endN_int.rename(columns={'name': 'end_neigh'})

  del df_endN_int['index_right']

  df = df_endN_int

  # list_cap = ['Sutro Heights','Outer Richmond','Cayuga','Mission Terace','Excelsior','University Mound','Portola','McLaren Park','Sunnydale',
  #           'Visitacion Valley','Candlestick Point SRA','Hunters Point','Bayview','India Bassin','Silver Terrace',
  #           'Apparel City','Produce Market','Bret Harte']

  # list_exemption = ['Seacliff','Outer Richmond','University Mound','Portola','Sunnydale','Visitacion Valley',
  #             'Candlestick Point SRA','Hunters Point','Apparel City','Bernal Heights',
  #             'Holly Park','Excelsior','McLaren Park'] ## Approximation from the service maps shown on Bay Wheels

  # df.loc[df['end_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  # df.loc[df['start_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  # df.loc[df['end_neigh'].isin(list_exemption), 'ebike_exemption'] = 'Yes'
  # df['ebike_cap'] = df['ebike_cap'].fillna(value='No')
  # df['ebike_exemption'] = df['ebike_exemption'].fillna(value='No')

  return df

In [6]:
df19 = pythonicGIS(df19)
display(df19.shape)
df20 = pythonicGIS(df20)
display(df20.shape)
df21 = pythonicGIS(df21)
display(df21.shape)

(1852712, 30)

(1732467, 40)

(1253169, 29)

### Estimating Revenue

This is where things got dense, since the fees changed in mid-2020. Therefore, I couldn't simply run a single helper function, but had to tailor for "pre-2020" and "post-2020" fee changes.
---
#### 2019 Pricing Scheme

Bay Wheels' rates for e-bikes changed on [March 2nd, 2020](https://medium.com/@baywheels/sf-ebike-pricing-details-1b97667f1e5a), from the [2019](https://www.tunneltime.io/san-francisco-usa/bay-wheels) rates:

<img width="200" height="183" src="drive/MyDrive/Gcolab/2019pricing.png">

Bay Wheels also include an annual [Bike Share for All](https://www.lyft.com/bikes/bay-wheels/bike-share-for-all) (BSFA) membership plan for low-income Bay Area residents.
---
#### 2020 Pricing Scheme

Bay Wheels' rates for e-bikes changed on [March 2nd, 2020](https://medium.com/@baywheels/sf-ebike-pricing-details-1b97667f1e5a), to the [following](https://www.lyft.com/bikes/bay-wheels/pricing):

<img width="200" height="183" src="drive/MyDrive/Gcolab/2020pricing.png">


Bay Wheels also include an annual [Bike Share for All](https://www.lyft.com/bikes/bay-wheels/bike-share-for-all) (BSFA) membership plan for low-income Bay Area residents.

#### Helper Functions: estimating trip revenue

In [7]:
## 2019 fare estimates
def revenueEST_2019(df):

    conditions  = [
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] <= 1800) 
        & (df['bike_share_for_all_trip'] == 'No') # 324360
        ),
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] > 1800)
        & (df['bike_share_for_all_trip'] == 'No') # 48725
        ),
        (
        (df['bike_share_for_all_trip'] == 'Yes') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] <= 3600)
        ), # 57049
        (
        (df['bike_share_for_all_trip'] == 'Yes') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] > 3600)
        ), # 784
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 900)
        & (df['bike_share_for_all_trip'] == 'No') # 5501
        ),
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 900)
        & (df['bike_share_for_all_trip'] == 'No') # 1466
        ),
        (
        (df['bike_share_for_all_trip'] == 'Yes') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 3600) # 0
        ), #0
        (
        (df['bike_share_for_all_trip'] == 'Yes') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 3600) # 0
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] <= 2700) 
        & (df['bike_share_for_all_trip'] == 'No') # 1355375
        ), 
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'classic_bike') & (df['duration_sec'] > 2700)
        & (df['bike_share_for_all_trip'] == 'No') # 7588
        ), 
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 2700)
        & (df['bike_share_for_all_trip'] == 'No') # 50255
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 2700)
        & (df['bike_share_for_all_trip'] == 'No') # 1180
        ),
    ]
  
    overhead_base_customer_classicBike1 = 2
    overhead_time_customer_classicBike2 = 3 * (abs(((df['duration_sec'] // 900)) - 2))
    overhead_base_BSFA_classicBike3 = 0
    overhead_time_BSFA_classicBike4 = 2 * (abs(((df['duration_sec'] // 900)) - 4))
    overhead_base_customer_ebike5 = 2 # $2 for the firt 15 min.
    overhead_time_customer_ebike6   = 2 + (3 * (abs(((df['duration_sec'] // 900)) - 1))) # $2 + $3/15 min.
    overhead_base_BSFA_ebike_NoCaps15 = 0 # No charges within the first 60 mins
    overhead_time_BSFA_ebike_NoCaps16 = (3 * (abs(((df['duration_sec'] // 900)) - 4))) # $3 every 15 mins. after 60 mins.
  
    overhead_base_subscriber_classicBike17 = 0
    overhead_time_subscriber_classicBike18 = 3 * (abs(((df['duration_sec'] // 900)) - 3))
  
    overhead_base_subscriber_ebike_NoCaps_ParkingFee25 = 0 # No charges within the first 45 mins
  
    overhead_time_subscriber_ebike_NoCaps_ParkingFee26 = (3 * (abs(((df['duration_sec'] // 900)) - 3))) # $3 every 15 mins. after 45 mins.
  
    choices = [
    overhead_base_customer_classicBike1,overhead_time_customer_classicBike2,overhead_base_BSFA_classicBike3,overhead_time_BSFA_classicBike4,
    overhead_base_customer_ebike5,overhead_time_customer_ebike6,
    overhead_base_BSFA_ebike_NoCaps15,overhead_time_BSFA_ebike_NoCaps16,
    overhead_base_subscriber_classicBike17,overhead_time_subscriber_classicBike18,
    overhead_base_subscriber_ebike_NoCaps_ParkingFee25,overhead_time_subscriber_ebike_NoCaps_ParkingFee26
    ]
  
    df["est_cost"] = np.select(conditions, choices, default=0).round(2)
  
  
    return df
####################################################################################################################################################################
####################################################################################################################################################################
## pre-2020 fare estimates
def revenueEST_pre2020(df):

  df_oldrates = df[df['start_time'].dt.strftime('%Y-%m-%d') < '2020-03-02']

  df_oldrates.loc[df_oldrates['trip_type'] == 'Docked-to-Docked', 'rideable_type'] = 'classic_bike'
  df_oldrates.loc[df_oldrates['trip_type'] != 'Docked-to-Docked', 'rideable_type'] = 'electric_bike'

  conditions  = [
      (
      (df_oldrates['user_type'] == 'Customer') & (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] <= 1800) 
      ),
      (
      (df_oldrates['user_type'] == 'Customer') & (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] > 1800)
      ),
      (
      (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] <= 3600)
      ), # 57049
      (
      (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] > 3600)
      ), # 784
      (
      (df_oldrates['user_type'] == 'Customer') & (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] <= 900)
      ),
      (
      (df_oldrates['user_type'] == 'Customer') & (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] > 900)
      ),
      (
      (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] <= 3600) # 0
      ), #0
      (
      (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] > 3600) # 0
      ),
      (
      (df_oldrates['user_type'] == 'Subscriber') & (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] <= 2700) 
      ), 
      (
      (df_oldrates['user_type'] == 'Subscriber') & (df_oldrates['rideable_type'] == 'classic_bike') & (df_oldrates['duration_sec'] > 2700)
      ), 
      (
      (df_oldrates['user_type'] == 'Subscriber') & (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] <= 2700)
      ),
      (
      (df_oldrates['user_type'] == 'Subscriber') & (df_oldrates['rideable_type'] == 'electric_bike') & (df_oldrates['duration_sec'] > 2700)
      ),
  ]

  overhead_base_customer_classicBike1 = 2
  overhead_time_customer_classicBike2 = 3 * (abs(((df_oldrates['duration_sec'] // 900)) - 2))
  overhead_base_BSFA_classicBike3 = 0
  overhead_time_BSFA_classicBike4 = 2 * (abs(((df_oldrates['duration_sec'] // 900)) - 4))
  overhead_base_customer_ebike5 = 2 # $2 for the firt 15 min.
  overhead_time_customer_ebike6   = 2 + (3 * (abs(((df_oldrates['duration_sec'] // 900)) - 1))) # $2 + $3/15 min.
  overhead_base_BSFA_ebike_NoCaps15 = 0 # No charges within the first 60 mins
  overhead_time_BSFA_ebike_NoCaps16 = (3 * (abs(((df_oldrates['duration_sec'] // 900)) - 4))) # $3 every 15 mins. after 60 mins.

  overhead_base_subscriber_classicBike17 = 0
  overhead_time_subscriber_classicBike18 = 3 * (abs(((df_oldrates['duration_sec'] // 900)) - 3))

  overhead_base_subscriber_ebike_NoCaps_ParkingFee25 = 0 # No charges within the first 45 mins

  overhead_time_subscriber_ebike_NoCaps_ParkingFee26 = (3 * (abs(((df_oldrates['duration_sec'] // 900)) - 3))) # $3 every 15 mins. after 45 mins.

  choices = [
  overhead_base_customer_classicBike1,overhead_time_customer_classicBike2,overhead_base_BSFA_classicBike3,overhead_time_BSFA_classicBike4,
  overhead_base_customer_ebike5,overhead_time_customer_ebike6,
  overhead_base_BSFA_ebike_NoCaps15,overhead_time_BSFA_ebike_NoCaps16,
  overhead_base_subscriber_classicBike17,overhead_time_subscriber_classicBike18,
  overhead_base_subscriber_ebike_NoCaps_ParkingFee25,overhead_time_subscriber_ebike_NoCaps_ParkingFee26
  ]

  df_oldrates["est_cost"] = np.select(conditions, choices, default=0).round(2)

  return df_oldrates
####################################################################################################################################################################
####################################################################################################################################################################
## post-2020 fare estimates
def revenueEST_post2020(df):

  df_newrates = df[df['start_time'].dt.strftime('%Y-%m-%d') >= '2020-03-02']

  df_newrates.loc[(df_newrates['trip_type'] == 'Dock-to-Dock') & (df_newrates['rideable_type'].isnull()),'rideable_type'] = 'classic_bike'
  df_newrates.loc[(df_newrates['trip_type'] != 'Dock-to-Dock') & (df_newrates['rideable_type'].isnull()),'rideable_type'] = 'electric_bike'

  list_cap = ['Sutro Heights','Outer Richmond','Cayuga','Mission Terace','Excelsior','University Mound','Portola','McLaren Park','Sunnydale',
         'Visitacion Valley','Candlestick Point SRA','Hunters Point','Bayview','India Bassin','Silver Terrace',
         'Apparel City','Produce Market','Bret Harte']

  list_exemption = ['Seacliff','Outer Richmond','University Mound','Portola','Sunnydale','Visitacion Valley',
            'Candlestick Point SRA','Hunters Point','Apparel City','Bernal Heights',
            'Holly Park','Excelsior','McLaren Park'] ## Approximation from the service maps shown on Bay Wheels

  df_newrates.loc[df_newrates['end_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  df_newrates.loc[df_newrates['start_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  df_newrates.loc[df_newrates['end_neigh'].isin(list_exemption), 'ebike_exemption'] = 'Yes'
  df_newrates['ebike_cap'] = df_newrates['ebike_cap'].fillna(value='No')
  df_newrates['ebike_exemption'] = df_newrates['ebike_exemption'].fillna(value='No')
  df_newrates['is_equity'] = df_newrates['is_equity'].fillna(value=True)
            
  conditions  = [
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] <= 1800) 
      & (df_newrates['is_equity'] == False) # 324360
      ),
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] > 1800)
      & (df_newrates['is_equity'] == False) # 48725
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] <= 3600)
      ), # 57049
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] > 3600)
      ), # 784
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 1800)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_exemption'] == 'Yes') # 102
      ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 1800)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_exemption'] == 'Yes') # 26
      ), ## !! ORIGINAL QUERY (EDITED ABOVE) !!   
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 1800)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_exemption'] == 'No') 
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) # 5011
      ),
      (
      (df_newrates['user_type'] == 'Customer') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 1800)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_exemption'] == 'No') 
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) # 1331
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 3600)
      & (df_newrates['ebike_cap'] == 'Yes')  & (df_newrates['ebike_exemption'] == 'Yes') # 0
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 3600)
      & (df_newrates['ebike_cap'] == 'Yes')  & (df_newrates['ebike_exemption'] == 'Yes') # 0
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 3600)
      & (df_newrates['ebike_cap'] == 'Yes')  & (df_newrates['ebike_exemption'] == 'No')  # 0
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 3600)
      & (df_newrates['ebike_cap'] == 'Yes')  & (df_newrates['ebike_exemption'] == 'No')  # 0
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 3600)
      & (df_newrates['ebike_cap'] == 'No')  & (df_newrates['ebike_exemption'] == 'No')  # 0
      ),
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 3600)
      & (df_newrates['ebike_cap'] == 'No')  & (df_newrates['ebike_exemption'] == 'No') # 0
      ),   
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 3600)
      & (df_newrates['ebike_cap'] == 'No')  & (df_newrates['ebike_exemption'] == 'Yes') # 0
      ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
      (
      (df_newrates['is_equity'] == True) & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 3600)
      & (df_newrates['ebike_cap'] == 'No')  & (df_newrates['ebike_exemption'] == 'Yes')  # 0
      ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] <= 2700) 
      & (df_newrates['is_equity'] == False) # 1355375
      ), 
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] != 'electric_bike') & (df_newrates['duration_sec'] > 2700)
      & (df_newrates['is_equity'] == False) # 7588
      ),   
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'Yes') & (df_newrates['ebike_exemption'] == 'Yes') #160
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'Yes') & (df_newrates['ebike_exemption'] == 'Yes') # 19
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'Yes') & (df_newrates['ebike_exemption'] == 'No')
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) # 358
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'Yes') & (df_newrates['ebike_exemption'] == 'No')
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) # 24
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'No') & (df_newrates['ebike_exemption'] == 'Yes') # 573
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'No') & (df_newrates['ebike_exemption'] == 'Yes') #11
      ),
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] <= 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'No') & (df_newrates['ebike_exemption'] == 'No')
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) #39428
      ),  ## !! ORIGINAL QUERY (EDITED ABOVE) !!
      (
      (df_newrates['user_type'] == 'Subscriber') & (df_newrates['rideable_type'] == 'electric_bike') & (df_newrates['duration_sec'] > 2700)
      & (df_newrates['is_equity'] == False) & (df_newrates['ebike_cap'] == 'No') & (df_newrates['ebike_exemption'] == 'No')
      & ((df_newrates['trip_type'] == 'Dockless_to_Dockless') | (df_newrates['trip_type'] == 'Docked-to-Dockless')) # 1025
      ),  ## !! ORIGINAL QUERY (EDITED ABOVE) !!
  ]

  overhead_base_customer_classicBike1 = 2 # 573384
  overhead_time_customer_classicBike2 = 3 * (abs(((df_newrates['duration_sec'] // 900)) - 2)) # 2027670
  overhead_base_BSFA_classicBike3 = 0
  overhead_time_BSFA_classicBike4 = 2 * (abs(((df_newrates['duration_sec'] // 900)) - 4))
  overhead_base_customer_ebike5 = 2 + (0.20 * (df_newrates['duration_sec'] / 60)) # Post-March 2020 Pricing Scheme
  overhead_time_customer_ebike6   = 5 + (3 * (abs(((df_newrates['duration_sec'] // 900)) - 2))) # Post-March 2020 Pricing Scheme # A maximum of $5 can be charged within 30 mins.
  overhead_base_customer_ebike_ParkingFee7 = 4 + (0.20 * (df_newrates['duration_sec'] / 60)) # Parking Fee adds a $2 surcharge
  overhead_time_customer_ebike_ParkingFee8  = 7 + (3 * (abs(((df_newrates['duration_sec'] // 900)) - 2))) # Parking Fee adds a $2 surcharge
  overhead_base_BSFA_ebike_Capped9 = 0.05 * (np.clip(df_newrates['duration_sec'],0,1200)) # Capped at $1, at a rate of 0.05 cent/min., that represents 20 minutes, or 600 seconds
  overhead_time_BSFA_ebike_Capped10 = 1 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 4))) # Capped at $1
  overhead_base_BSFA_ebike_Capped_ParkingFee11 = 2 + (0.05 * ((np.clip(df_newrates['duration_sec'],0,1200)) / 60)) # Parking Fee adds a $2 surcharge
  overhead_time_BSFA_ebike_Capped_ParkingFee12 = 3 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 4))) # Parking Fee adds a $2 surcharge
  overhead_base_BSFA_ebike_NoCaps_ParkingFee13 = 2 + (0.05 * (df_newrates['duration_sec'] / 60))
  overhead_time_BSFA_ebike_NoCaps_ParkingFee14 = 5 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 4))) # Parking Fee adds a $2 surcharge + the maximum riding fee that can be charged within 1 hours with no caps is $3
  overhead_base_BSFA_ebike_NoCaps15 = (0.05 * (df_newrates['duration_sec'] / 60)) # Post-March 2020 Pricing Scheme
  overhead_time_BSFA_ebike_NoCaps16 = 3 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 4))) # Post-March 2020 Pricing Scheme
  overhead_base_subscriber_classicBike17 = 0
  overhead_time_subscriber_classicBike18 = 3 * (abs(((df_newrates['duration_sec'] // 900)) - 3))
  overhead_base_subscriber_ebike_Capped19 = 0.15 * ((np.clip(df_newrates['duration_sec'],0,800) / 60)) # Capped at $2, at a rate of 0.15 cent/min., that represents approx. 13.3 minutes, or rounded to 800 seconds
  overhead_time_subscriber_ebike_Capped20 = 2 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 3))) # Capped at $2 within the first 45 mins.
  overhead_base_subscriber_ebike_Capped_ParkingFee21 = 2 + (0.15 * ((np.clip(df_newrates['duration_sec'],0,800) / 60)))
  overhead_time_subscriber_ebike_Capped_ParkingFee22 = 4 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 3))) # Parking Fee adds a $2 surcharge
  overhead_base_subscriber_ebike_NoCaps23 = 0.15 * ((df_newrates['duration_sec'] / 60))
  overhead_time_subscriber_ebike_NoCaps24 = 6.75 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 3))) # A maximum of $6.75 can be charged within 45 mins
  overhead_base_subscriber_ebike_NoCaps_ParkingFee25 = 2 + (0.15 * ((df_newrates['duration_sec'] / 60))) # Post-March 2020 Pricing Scheme # Parking Fee adds a $2 surcharge
  overhead_time_subscriber_ebike_NoCaps_ParkingFee26 = 8.75 + (2 * (abs(((df_newrates['duration_sec'] // 900)) - 3))) # Post-March 2020 Pricing Scheme # Parking Fee adds a $2 surcharge 

  choices = [
  overhead_base_customer_classicBike1,overhead_time_customer_classicBike2,overhead_base_BSFA_classicBike3,overhead_time_BSFA_classicBike4,
  overhead_base_customer_ebike5,overhead_time_customer_ebike6,
  overhead_base_customer_ebike_ParkingFee7,overhead_time_customer_ebike_ParkingFee8,
  overhead_base_BSFA_ebike_Capped9,overhead_time_BSFA_ebike_Capped10,overhead_base_BSFA_ebike_Capped_ParkingFee11,
  overhead_time_BSFA_ebike_Capped_ParkingFee12,overhead_base_BSFA_ebike_NoCaps_ParkingFee13,overhead_time_BSFA_ebike_NoCaps_ParkingFee14,
  overhead_base_BSFA_ebike_NoCaps15,overhead_time_BSFA_ebike_NoCaps16,
  overhead_base_subscriber_classicBike17,overhead_time_subscriber_classicBike18,
  overhead_base_subscriber_ebike_Capped19,overhead_time_subscriber_ebike_Capped20,overhead_base_subscriber_ebike_Capped_ParkingFee21,
  overhead_time_subscriber_ebike_Capped_ParkingFee22,overhead_base_subscriber_ebike_NoCaps23,overhead_time_subscriber_ebike_NoCaps24,
  overhead_base_subscriber_ebike_NoCaps_ParkingFee25,overhead_time_subscriber_ebike_NoCaps_ParkingFee26
  ]

  df_newrates["est_cost"] = np.select(conditions, choices, default=0).round(2)

  return df_newrates
####################################################################################################################################################################
####################################################################################################################################################################
## Nearly identical to the post-2020 fare estimates function, but with the "equity" variable removed.
def revenueEST_2021(df):  

  list_cap = ['Sutro Heights','Outer Richmond','Cayuga','Mission Terace','Excelsior','University Mound','Portola','McLaren Park','Sunnydale',
         'Visitacion Valley','Candlestick Point SRA','Hunters Point','Bayview','India Bassin','Silver Terrace',
         'Apparel City','Produce Market','Bret Harte']

  list_exemption = ['Seacliff','Outer Richmond','University Mound','Portola','Sunnydale','Visitacion Valley',
            'Candlestick Point SRA','Hunters Point','Apparel City','Bernal Heights',
            'Holly Park','Excelsior','McLaren Park'] ## Approximation from the service maps shown on Bay Wheels

  df.loc[df['end_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  df.loc[df['start_neigh'].isin(list_cap), 'ebike_cap'] = 'Yes'
  df.loc[df['end_neigh'].isin(list_exemption), 'ebike_exemption'] = 'Yes'
  df['ebike_cap'] = df['ebike_cap'].fillna(value='No')
  df['ebike_exemption'] = df['ebike_exemption'].fillna(value='No')


  conditions  = [
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] <= 1800) 
        ),
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] > 1800)
        ),
        (
        (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] <= 3600)
        ), # 57049
        (
        (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] > 3600)
        ), # 784
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 1800)
        & (df['ebike_exemption'] == 'Yes') # 102
        ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 1800)
        & (df['ebike_exemption'] == 'Yes') # 26
        ), ## !! ORIGINAL QUERY (EDITED ABOVE) !!   
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 1800)
        & (df['ebike_exemption'] == 'No') 
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) # 5011
        ),
        (
        (df['user_type'] == 'Customer') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 1800)
        & (df['ebike_exemption'] == 'No') 
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) # 1331
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 3600)
        & (df['ebike_cap'] == 'Yes')  & (df['ebike_exemption'] == 'Yes') # 0
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 3600)
        & (df['ebike_cap'] == 'Yes')  & (df['ebike_exemption'] == 'Yes') # 0
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 3600)
        & (df['ebike_cap'] == 'Yes')  & (df['ebike_exemption'] == 'No')  # 0
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 3600)
        & (df['ebike_cap'] == 'Yes')  & (df['ebike_exemption'] == 'No')  # 0
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 3600)
        & (df['ebike_cap'] == 'No')  & (df['ebike_exemption'] == 'No')  # 0
        ),
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 3600)
        & (df['ebike_cap'] == 'No')  & (df['ebike_exemption'] == 'No') # 0
        ),   
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 3600)
        & (df['ebike_cap'] == 'No')  & (df['ebike_exemption'] == 'Yes') # 0
        ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
        (
        (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 3600)
        & (df['ebike_cap'] == 'No')  & (df['ebike_exemption'] == 'Yes')  # 0
        ), ## !! ORIGINAL QUERY (EDITED ABOVE) !! 
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] <= 2700) 
        ), 
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] != 'electric_bike') & (df['duration_sec'] > 2700)
        ),   
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 2700)
        & (df['ebike_cap'] == 'Yes') & (df['ebike_exemption'] == 'Yes') #160
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 2700)
        & (df['ebike_cap'] == 'Yes') & (df['ebike_exemption'] == 'Yes') # 19
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 2700)
        & (df['ebike_cap'] == 'Yes') & (df['ebike_exemption'] == 'No')
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) # 358
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 2700)
        & (df['ebike_cap'] == 'Yes') & (df['ebike_exemption'] == 'No')
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) # 24
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 2700)
        & (df['ebike_cap'] == 'No') & (df['ebike_exemption'] == 'Yes') # 573
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 2700)
        & (df['ebike_cap'] == 'No') & (df['ebike_exemption'] == 'Yes') #11
        ),
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] <= 2700)
        & (df['ebike_cap'] == 'No') & (df['ebike_exemption'] == 'No')
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) #39428
        ),  ## !! ORIGINAL QUERY (EDITED ABOVE) !!
        (
        (df['user_type'] == 'Subscriber') & (df['rideable_type'] == 'electric_bike') & (df['duration_sec'] > 2700)
        & (df['ebike_cap'] == 'No') & (df['ebike_exemption'] == 'No')
        & ((df['trip_type'] == 'Dockless_to_Dockless') | (df['trip_type'] == 'Docked-to-Dockless')) # 1025
        ),  ## !! ORIGINAL QUERY (EDITED ABOVE) !!
    ]
  
  overhead_base_customer_classicBike1 = 2 # 573384
  overhead_time_customer_classicBike2 = 3 * (abs(((df['duration_sec'] // 900)) - 2)) # 2027670
  overhead_base_BSFA_classicBike3 = 0
  overhead_time_BSFA_classicBike4 = 2 * (abs(((df['duration_sec'] // 900)) - 4))
  overhead_base_customer_ebike5 = 2 + (0.20 * (df['duration_sec'] / 60)) # Post-March 2020 Pricing Scheme
  overhead_time_customer_ebike6   = 5 + (3 * (abs(((df['duration_sec'] // 900)) - 2))) # Post-March 2020 Pricing Scheme # A maximum of $5 can be charged within 30 mins.
  overhead_base_customer_ebike_ParkingFee7 = 4 + (0.20 * (df['duration_sec'] / 60)) # Parking Fee adds a $2 surcharge
  overhead_time_customer_ebike_ParkingFee8  = 7 + (3 * (abs(((df['duration_sec'] // 900)) - 2))) # Parking Fee adds a $2 surcharge
  overhead_base_BSFA_ebike_Capped9 = 0.05 * (np.clip(df['duration_sec'],0,1200)) # Capped at $1, at a rate of 0.05 cent/min., that represents 20 minutes, or 600 seconds
  overhead_time_BSFA_ebike_Capped10 = 1 + (2 * (abs(((df['duration_sec'] // 900)) - 4))) # Capped at $1
  overhead_base_BSFA_ebike_Capped_ParkingFee11 = 2 + (0.05 * ((np.clip(df['duration_sec'],0,1200)) / 60)) # Parking Fee adds a $2 surcharge
  overhead_time_BSFA_ebike_Capped_ParkingFee12 = 3 + (2 * (abs(((df['duration_sec'] // 900)) - 4))) # Parking Fee adds a $2 surcharge
  overhead_base_BSFA_ebike_NoCaps_ParkingFee13 = 2 + (0.05 * (df['duration_sec'] / 60))
  overhead_time_BSFA_ebike_NoCaps_ParkingFee14 = 5 + (2 * (abs(((df['duration_sec'] // 900)) - 4))) # Parking Fee adds a $2 surcharge + the maximum riding fee that can be charged within 1 hours with no caps is $3
  overhead_base_BSFA_ebike_NoCaps15 = (0.05 * (df['duration_sec'] / 60)) # Post-March 2020 Pricing Scheme
  overhead_time_BSFA_ebike_NoCaps16 = 3 + (2 * (abs(((df['duration_sec'] // 900)) - 4))) # Post-March 2020 Pricing Scheme
  overhead_base_subscriber_classicBike17 = 0
  overhead_time_subscriber_classicBike18 = 3 * (abs(((df['duration_sec'] // 900)) - 3))
  overhead_base_subscriber_ebike_Capped19 = 0.15 * ((np.clip(df['duration_sec'],0,800) / 60)) # Capped at $2, at a rate of 0.15 cent/min., that represents approx. 13.3 minutes, or rounded to 800 seconds
  overhead_time_subscriber_ebike_Capped20 = 2 + (2 * (abs(((df['duration_sec'] // 900)) - 3))) # Capped at $2 within the first 45 mins.
  overhead_base_subscriber_ebike_Capped_ParkingFee21 = 2 + (0.15 * ((np.clip(df['duration_sec'],0,800) / 60)))
  overhead_time_subscriber_ebike_Capped_ParkingFee22 = 4 + (2 * (abs(((df['duration_sec'] // 900)) - 3))) # Parking Fee adds a $2 surcharge
  overhead_base_subscriber_ebike_NoCaps23 = 0.15 * ((df['duration_sec'] / 60))
  overhead_time_subscriber_ebike_NoCaps24 = 6.75 + (2 * (abs(((df['duration_sec'] // 900)) - 3))) # A maximum of $6.75 can be charged within 45 mins
  overhead_base_subscriber_ebike_NoCaps_ParkingFee25 = 2 + (0.15 * ((df['duration_sec'] / 60))) # Post-March 2020 Pricing Scheme # Parking Fee adds a $2 surcharge
  overhead_time_subscriber_ebike_NoCaps_ParkingFee26 = 8.75 + (2 * (abs(((df['duration_sec'] // 900)) - 3))) # Post-March 2020 Pricing Scheme # Parking Fee adds a $2 surcharge 
  
  choices = [
  overhead_base_customer_classicBike1,overhead_time_customer_classicBike2,overhead_base_BSFA_classicBike3,overhead_time_BSFA_classicBike4,
  overhead_base_customer_ebike5,overhead_time_customer_ebike6,
  overhead_base_customer_ebike_ParkingFee7,overhead_time_customer_ebike_ParkingFee8,
  overhead_base_BSFA_ebike_Capped9,overhead_time_BSFA_ebike_Capped10,overhead_base_BSFA_ebike_Capped_ParkingFee11,
  overhead_time_BSFA_ebike_Capped_ParkingFee12,overhead_base_BSFA_ebike_NoCaps_ParkingFee13,overhead_time_BSFA_ebike_NoCaps_ParkingFee14,
  overhead_base_BSFA_ebike_NoCaps15,overhead_time_BSFA_ebike_NoCaps16,
  overhead_base_subscriber_classicBike17,overhead_time_subscriber_classicBike18,
  overhead_base_subscriber_ebike_Capped19,overhead_time_subscriber_ebike_Capped20,overhead_base_subscriber_ebike_Capped_ParkingFee21,
  overhead_time_subscriber_ebike_Capped_ParkingFee22,overhead_base_subscriber_ebike_NoCaps23,overhead_time_subscriber_ebike_NoCaps24,
  overhead_base_subscriber_ebike_NoCaps_ParkingFee25,overhead_time_subscriber_ebike_NoCaps_ParkingFee26
  ]
  
  df["est_cost"] = np.select(conditions, choices, default=0).round(2)
  
  return df

In [8]:
df2019 = revenueEST_2019(df19)
print(df2019.est_cost.sum().round(2)) # $1,307,108
##################################
df20_old = revenueEST_pre2020(df20)
display(df20_old["est_cost"].sum()) # $535,392
df20_new = revenueEST_post2020(df20)
display(df20_new["est_cost"].sum()) # $3,589,525 // 3867352.03
df2020 = df20_old.append(df20_new, ignore_index=True)
print(df2020.est_cost.sum().round(2)) # $4,124,917 // 4,402,744.03
##################################
df2021 = revenueEST_2021(df21)
print(df2021.est_cost.sum().round(2)) # $4,293,155

1307108


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


535392.0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


3867352.03

4402744.03
4293154.94


#### Save to CSV

In [9]:
# from google.colab import drive
# drive.mount('/drive')

# df2019.to_csv('/drive/MyDrive/Gcolab/modifiedData/lyft/lyft2019.csv',index=False)
# df2020.to_csv('/drive/MyDrive/Gcolab/modifiedData/lyft/lyft2020.csv',index=False)
# df2021.to_csv('/drive/MyDrive/Gcolab/modifiedData/lyft/lyft2021.csv',index=False)

Mounted at /drive


## One CSV to rule them all

In [2]:
# Import the raw csv yearly datasets into their respective pandas dataframe
from google.colab import drive
drive.mount('/content/drive')

df2019 = pd.read_csv(r'drive/MyDrive/Gcolab/modifiedData/lyft/lyft2019.csv',float_precision=None)
display(df2019.shape)
df2020 = pd.read_csv(r'drive/MyDrive/Gcolab/modifiedData/lyft/lyft2020.csv',float_precision=None)
display(df2020.shape)
df2021 = pd.read_csv(r'drive/MyDrive/Gcolab/modifiedData/lyft/lyft2021.csv',float_precision=None)
display(df2021.shape)

dfX = df2020.append(df2019, ignore_index=True)
df = df2021.append(dfX, ignore_index=True)

display(df.shape)
# (4838348, 44)

Mounted at /content/drive


(1852712, 30)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,3,5,6,7,17,18,19,22,23,26,27,30,31,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1732467, 42)

(1253169, 32)

(4838348, 44)

##### Combine CSVs &Ad Hoc Cleaning

In [3]:
del df['ride_id']
del df['member_casual']
del df['fname']

columns = ['start_station_name', 'start_station_id','end_station_name','end_station_id','bike_id','rental_access_method','bike_share_for_all_trip','is_equity']
# 'bike_id','rental_access_method','bike_share_for_all_trip','is_equity' are deprecated fields; I'm retaining those for partial/ad hoc analysis.
df[columns] = df.loc[:, columns].fillna("")
df = df.dropna(axis=1)

def convert2datetime(dataframe,columns): 
    for column in columns:
        dataframe[column] = pd.to_datetime(dataframe[column],dayfirst=True)       
        dataframe[column + str('_day')] = dataframe[column].dt.day_name()
        dataframe[column + str('_month')] = dataframe[column].dt.month_name()
        dataframe[column + str('_hour')] = dataframe[column].dt.round('H').dt.hour
        dataframe[column + str('_year')] = dataframe[column].dt.year

    return dataframe.dtypes

cols = ['start_time','end_time']

convert2datetime(df,cols)

df['Quarter'] = df['start_time'].dt.to_period("Q")

df['route_neigh'] = "From " + df['start_neigh'] + " To " + df['end_neigh']

df['rideable_type'] = df['rideable_type'].replace({'classic_bike': 'docked_bike'})

df = df.sort_values(by='start_time')

lon = df[['start_station_longitude','end_station_longitude']].values
lat = df[['start_station_latitude','end_station_latitude']].values

from pyproj import Transformer # convert coords to UTM to get distance in meters.
# https://gis.stackexchange.com/questions/334271/converting-large-data-with-lat-and-long-into-x-and-y
trans = Transformer.from_crs(
    "epsg:4326",
    "+proj=utm +zone=10 +ellps=WGS84",
    always_xy=True,
)
lon, lat = trans.transform(lon, lat)
df[['start_lon_utm','end_lon_utm']] = lon
df[['start_lat_utm','end_lat_utm']] = lat

x1 = df.start_lon_utm.values
x2 = df.end_lon_utm.values

y1 = df.start_lat_utm.values
y2 = df.end_lat_utm.values

df['eucl_dist_m'] = np.sqrt( np.square( x2 - x1 )  + np.square( y2 - y1 ) ).round(0)

del df['start_lat_utm']
del df['end_lat_utm']
del df['start_lon_utm']
del df['end_lon_utm']

## Datapoints that didn't get a match with ID attributes, but are valid (slight difference in naming conventions of stations)
replace_values = {'Steuart St at Market St' : 'Market St at Steuart St',
                 'Broadway at Kearny':'Broadway at Kearny St',
                  'Mendall St at Fairfax Ave':'Mendell St at Fairfax Ave',
                  'Green St at Van Ness St':'Green St at Van Ness Ave',
                  'Clement St at 32nd Avenue':'Clement St at 32nd Ave',
                  '5th St at Folsom':'Folsom St at 5th St',
                  'San Francisco Caltrain Station 2  (Townsend St at 4th St)':'San Francisco Caltrain (Townsend St at 4th St)'
                 } 
df = df.replace({"start_station_name": replace_values}) 
df = df.replace({"end_station_name": replace_values}) 

# Stations in the dataset that are irrelevant to the analysis (Test sites etc.)
df = df[~df['start_station_name'].astype(str).isin(['Golden Gate Ave at Hyde St', '16th St Depot','SF Depot-2 (Minnesota St Outbound)',
                           '16th Depot Bike Fleet Station', '23rd St at San Bruno Ave','Howard workshop - Station in a Box',
                           'Outside Lands (Temporary Station)','Lab - Howard','SF STATION IN A BOX 1','Minnesota St Depot'])]
df = df[~df['end_station_name'].astype(str).isin(['Golden Gate Ave at Hyde St', '16th St Depot','SF Depot-2 (Minnesota St Outbound)',
                           '16th Depot Bike Fleet Station', '23rd St at San Bruno Ave','Howard workshop - Station in a Box',
                           'Outside Lands (Temporary Station)','Lab - Howard','SF STATION IN A BOX 1','Minnesota St Depot'])]
df = df[~df['start_station_id'].astype(str).isin(['33', '344', '449', '136', '428', '367'])]
df = df[~df['end_station_id'].astype(str).isin(['33', '344', '449', '136', '428', '367'])]


display(df.shape)

(4822411, 34)

##### Feature Engineering, Round 2

---
###### Merging dock station's attributes to the Master dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')
stationdf = pd.read_csv(r'drive/MyDrive/Gcolab/rawdata/DataSF/Bay_Area_Bikeshare_Stations.csv',float_precision=None,dtype={"Station ID DOMO": "string", "Station ID": "string"})

stationdf = stationdf[['Station ID','Station ID DOMO','Station Name','Has Kiosk','Dock Count','Station Latitude','Station Longitude']]

start_stationdf = stationdf.add_prefix('start_')
end_stationdf = stationdf.add_prefix('end_')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


What's happening here is that I am trying to add the docking station's information to the wrangled Lyft dataset. I first join the two dataset on a common ID, i.e. the station's name; where it gets tricky is that 1) I need to join separately for the start and end's station, and 2) station's name, although it joins the majority of the dataset, still does not capture all of the data. Therefore, the messy code below is essentially me taking subsets of the data that were not joined by station's name, and iterated upon by joining with variation of station's ID (again, the change in data schema between years in the raw Lyft system dataset did not do me any favors). Hence this was quite a tedious process, which I began by spittling the wrangled Lyft dataset by the trip_type category (Dockless-to-Dockless, Docked-to-Docked, Docked-to-Dockless, and Dockless-to-Docked...obviously the Dockless-to-Dockless subset did not need to be joined, and was appended last). 

So yes, there is a method to this madness, which took me a bit of time to work myself backward after I returned to this project after a few months on hold; perhaps this is revealing that I still have ways to write cleaner and more concise code. I mean, sure, I could have written a helper function and looped some of the batch processing that took place...but I was also double-checking every steps and tracking how much of the data was getting matches or dropped. Look it's not my best code OK but it got the job done and sometimes after spending a stupid amount of time working through a hydra-like problem just to join some bits of data to 4 million rows you know what I'll take it. Live and learn. Learn from my mistakes as I learn from mine. I don't know why I'm writing all this I guess it's a form of coping from something that should be called *post-mortem programmatis* or whatev. OK peace out.

In [5]:
# Merge the DATASF's docking dataset to the Master Lyft dataset; remove all dockless trips first, then re-append later.

# print(len(df.loc[df['trip_type'] != 'Dockless-to-Dockless']))
# print(len(df.loc[df['trip_type'] == 'Dockless-to-Dockless']))

df_dl2dl = df.loc[df['trip_type'] == 'Dockless-to-Dockless']
df_d2d = df.loc[df['trip_type'] == 'Docked-to-Docked']
df_d2dl = df.loc[df['trip_type'] == 'Docked-to-Dockless']
df_dl2d = df.loc[df['trip_type'] == 'Dockless-to-Docked']

In [7]:
# Match all d2d sets -- START

merged_df = df_d2d.merge(start_stationdf, left_on=['start_station_name'], right_on = ['start_Station Name'], how='left')
print(merged_df.shape) # 3467999 >> TOTAL ROWS
## start_merged_df will be appended.
start_merged_df = merged_df.dropna(axis=0, subset=['start_Station Name']) # append candidate 1, Merging on Start Station's Name
print(start_merged_df.shape) # 3425450 / 3467999 >> 98.7% total match >> TOTAL ROWS that were merged with start_stationdf
## start_merged_df2 will be appended.
start_notmerged_df = merged_df[merged_df['start_Station Name'].isnull()]
print(start_notmerged_df.shape) # 42549 >> 1% of the set (referred below as subset) unmatched >> TOTAL ROWS that were not merged with start_stationdf; everything below is to ensure that all possible matches (using different fields) are met.
start_notmerged_df = start_notmerged_df[start_notmerged_df.columns[:-7]]
start_notmerged_df['start_station_id'] = start_notmerged_df['start_station_id'].astype(str).str.replace('.0','')
merged_df2 = start_notmerged_df.merge(start_stationdf, left_on=['start_station_id'], right_on = ['start_Station ID'], how='left') # Merging on Start Station's unique ID
start_merged_df2 = merged_df2.dropna(axis=0, subset=['start_Station ID']) # append candidate 2
print(start_merged_df2.shape) # 27458 / 42549 >> 64.5% of the subset matched based on station ID field

start_notmerged_df2 = merged_df2[merged_df2['start_Station ID'].isnull()]
print(start_notmerged_df2.shape) # 15091 / 42549 >> Still 35% of the subset left to be ID.
start_notmerged_df2 = start_notmerged_df2[start_notmerged_df2.columns[:-7]]
merged_df3 = start_notmerged_df2.merge(start_stationdf, left_on=['start_station_id'], right_on = ['start_Station ID DOMO'], how='left') # Merging on Start Station's unique ID <> "ID Domo"
start_merged_df3 = merged_df3.dropna(axis=0, subset=['start_Station ID DOMO']) # append candidate 3, 430
print(start_merged_df3.shape) # 4191 / 15091 >> 28% of the subset id.

start_notmerged_df3 = merged_df3[merged_df3['start_Station ID DOMO'].isnull()]
print(start_notmerged_df3.shape) # 10900 rows left to be ID >> 0.003% of the d2d set Dropped
start_no_merges = start_notmerged_df3[start_notmerged_df3['start_station_id'].isnull()]
print(start_no_merges.shape) 

start_append1 = start_merged_df.append(start_merged_df2, ignore_index=True)
print(start_append1.shape) # 3452908
start_append2 = start_append1.append(start_merged_df3, ignore_index=True)
print(start_append2.shape) # 3457099
start_append3 = start_append2.append(start_notmerged_df3, ignore_index=True)
print(start_append3.shape) # 3457099
merged_df1 = start_append3.merge(end_stationdf, left_on=['end_station_name'], right_on = ['end_Station Name'], how='left')
print(merged_df1.shape) # 3467999
merged_df1 = merged_df1.dropna(axis=0, subset=['start_Dock Count','end_Dock Count'])
print(len(merged_df1)) # 98.5% of the d2d set has been matched.

### 
end_merged_df = merged_df1.dropna(axis=0, subset=['end_Station Name']) # append candidate 1, Merging on Start Station's Name

end_notmerged_df = merged_df1[merged_df1['end_Station Name'].isnull()]

end_notmerged_df = end_notmerged_df[end_notmerged_df.columns[:-7]]

end_notmerged_df['end_station_id'] = end_notmerged_df['end_station_id'].astype(str).str.replace('.0','')

merged_df2 = end_notmerged_df.merge(end_stationdf, left_on=['end_station_id'], right_on = ['end_Station ID'], how='left')

end_merged_df2 = merged_df2.dropna(axis=0, subset=['end_Station ID']) # append candidate 2, 30000

end_notmerged_df2 = merged_df2[merged_df2['end_Station ID'].isnull()]

end_notmerged_df2 = end_notmerged_df2[end_notmerged_df2.columns[:-7]]

merged_df3 = end_notmerged_df2.merge(end_stationdf, left_on=['end_station_id'], right_on = ['end_Station ID DOMO'], how='left')

end_merged_df3 = merged_df3.dropna(axis=0, subset=['end_Station ID DOMO']) # append candidate 3, 430

end_notmerged_df3 = merged_df3[merged_df3['end_Station ID DOMO'].isnull()]

end_notmerged_df3 = end_notmerged_df3[end_notmerged_df3['end_station_id'].isnull()]

end_no_merges = end_notmerged_df3[end_notmerged_df3['end_station_id'].isnull()]

end_append1 = end_merged_df.append(end_merged_df2, ignore_index=True)

end_append2 = end_append1.append(end_merged_df3, ignore_index=True)

merged_df1X = end_append2.append(end_notmerged_df3, ignore_index=True)

display(merged_df1X.shape)
### 3,416,602 / 3,467,999 rows

(3467999, 41)
(3425450, 41)
(42549, 41)
(27458, 41)
(15091, 41)
(4191, 41)
(10900, 41)
(0, 41)
(3452908, 41)
(3457099, 41)
(3467999, 41)
(3467999, 48)
3416602


(3416602, 48)

In [8]:
# Match all df_d2dl sets

merged_df = df_d2dl.merge(start_stationdf, left_on=['start_station_name'], right_on = ['start_Station Name'], how='left')
print(merged_df.shape) # 348636 >> TOTAL ROWS
## start_merged_df will be appended.
start_merged_df = merged_df.dropna(axis=0, subset=['start_Station Name']) # append candidate 1, Merging on Start Station's Name
print(start_merged_df.shape) # 344138 / 348636 >> 98.7% total match >> TOTAL ROWS that were merged with start_stationdf
## start_merged_df2 will be appended.
start_notmerged_df = merged_df[merged_df['start_Station Name'].isnull()]
print(start_notmerged_df.shape) # 42549 >> 1% of the set (referred below as subset) unmatched >> TOTAL ROWS that were not merged with start_stationdf; everything below is to ensure that all possible matches (using different fields) are met.
start_notmerged_df = start_notmerged_df[start_notmerged_df.columns[:-7]]
start_notmerged_df['start_station_id'] = start_notmerged_df['start_station_id'].astype(str).str.replace('.0','')
merged_df2 = start_notmerged_df.merge(start_stationdf, left_on=['start_station_id'], right_on = ['start_Station ID'], how='left') # Merging on Start Station's unique ID
start_merged_df2 = merged_df2.dropna(axis=0, subset=['start_Station ID']) # append candidate 2
print(start_merged_df2.shape) # 27458 / 42549 >> 64.5% of the subset matched based on station ID field

start_notmerged_df2 = merged_df2[merged_df2['start_Station ID'].isnull()]
print(start_notmerged_df2.shape) # 15091 / 42549 >> Still 35% of the subset left to be ID.
start_notmerged_df2 = start_notmerged_df2[start_notmerged_df2.columns[:-7]]
merged_df3 = start_notmerged_df2.merge(start_stationdf, left_on=['start_station_id'], right_on = ['start_Station ID DOMO'], how='left') # Merging on Start Station's unique ID <> "ID Domo"
start_merged_df3 = merged_df3.dropna(axis=0, subset=['start_Station ID DOMO']) # append candidate 3, 430
print(start_merged_df3.shape) # 4191 / 15091 >> 28% of the subset id.

start_notmerged_df3 = merged_df3[merged_df3['start_Station ID DOMO'].isnull()]
print(start_notmerged_df3.shape) # 10900 rows left to be ID >> 0.003% of the d2d set Dropped
start_no_merges = start_notmerged_df3[start_notmerged_df3['start_station_id'].isnull()]
print(start_no_merges.shape) 

start_append1 = start_merged_df.append(start_merged_df2, ignore_index=True)
print(start_append1.shape) # 3452908
start_append2 = start_append1.append(start_merged_df3, ignore_index=True)
print(start_append2.shape) # 3457099
start_append3 = start_append2.append(start_notmerged_df3, ignore_index=True)
print(start_append3.shape) # 3457099

merged_df2X = start_append3.dropna(axis=0, subset=['start_Dock Count'])
print(len(merged_df2X)) # 99% of the df_d2dl set has been matched.

(348636, 41)
(344138, 41)
(4498, 41)
(1685, 41)
(2813, 41)
(900, 41)
(1913, 41)
(0, 41)
(345823, 41)
(346723, 41)
(348636, 41)
346723


In [9]:
# Match all df_dl2d sets

merged_df = df_dl2d.merge(end_stationdf, left_on=['end_station_name'], right_on = ['end_Station Name'], how='left')
print(merged_df.shape) # 348636 >> TOTAL ROWS
## end_merged_df will be appended.
end_merged_df = merged_df.dropna(axis=0, subset=['end_Station Name']) # append candidate 1, Merging on end Station's Name
print(end_merged_df.shape) # 344138 / 348636 >> 98.7% total match >> TOTAL ROWS that were merged with end_stationdf
## end_merged_df2 will be appended.
end_notmerged_df = merged_df[merged_df['end_Station Name'].isnull()]
print(end_notmerged_df.shape) # 42549 >> 1% of the set (referred below as subset) unmatched >> TOTAL ROWS that were not merged with end_stationdf; everything below is to ensure that all possible matches (using different fields) are met.
end_notmerged_df = end_notmerged_df[end_notmerged_df.columns[:-7]]
end_notmerged_df['end_station_id'] = end_notmerged_df['end_station_id'].astype(str).str.replace('.0','')
merged_df2 = end_notmerged_df.merge(end_stationdf, left_on=['end_station_id'], right_on = ['end_Station ID'], how='left') # Merging on end Station's unique ID
end_merged_df2 = merged_df2.dropna(axis=0, subset=['end_Station ID']) # append candidate 2
print(end_merged_df2.shape) # 27458 / 42549 >> 64.5% of the subset matched based on station ID field

end_notmerged_df2 = merged_df2[merged_df2['end_Station ID'].isnull()]
print(end_notmerged_df2.shape) # 15091 / 42549 >> Still 35% of the subset left to be ID.
end_notmerged_df2 = end_notmerged_df2[end_notmerged_df2.columns[:-7]]
merged_df3 = end_notmerged_df2.merge(end_stationdf, left_on=['end_station_id'], right_on = ['end_Station ID DOMO'], how='left') # Merging on end Station's unique ID <> "ID Domo"
end_merged_df3 = merged_df3.dropna(axis=0, subset=['end_Station ID DOMO']) # append candidate 3, 430
print(end_merged_df3.shape) # 4191 / 15091 >> 28% of the subset id.

end_notmerged_df3 = merged_df3[merged_df3['end_Station ID DOMO'].isnull()]
print(end_notmerged_df3.shape) # 10900 rows left to be ID >> 0.003% of the d2d set Dropped
end_no_merges = end_notmerged_df3[end_notmerged_df3['end_station_id'].isnull()]
print(end_no_merges.shape) 

end_append1 = end_merged_df.append(end_merged_df2, ignore_index=True)
print(end_append1.shape) # 3452908
end_append2 = end_append1.append(end_merged_df3, ignore_index=True)
print(end_append2.shape) # 3457099
end_append3 = end_append2.append(end_notmerged_df3, ignore_index=True)
print(end_append3.shape) # 3457099

merged_df3X = end_append3.dropna(axis=0, subset=['end_Dock Count'])

print(len(merged_df3X)) # 318615/320062 of the df_dl2d set has been matched.

(320062, 41)
(316175, 41)
(3887, 41)
(1633, 41)
(2254, 41)
(807, 41)
(1447, 41)
(0, 41)
(317808, 41)
(318615, 41)
(320062, 41)
318615


In [10]:
final_append1 = merged_df1X.append(merged_df2X, ignore_index=True)
final_append2 = final_append1.append(merged_df3X, ignore_index=True)
df_master = final_append2.append(df_dl2dl, ignore_index=True)

display(df_master.shape)

(4767654, 48)

In [11]:
df = df_master

df.loc[df['trip_type'] == 'Docked-to-Docked','start_station_latitude'] = df['start_Station Latitude']
df.loc[df['trip_type'] == 'Docked-to-Docked','start_station_longitude'] = df['start_Station Longitude']
df.loc[df['trip_type'] == 'Docked-to-Docked','end_station_latitude'] = df['end_Station Latitude']
df.loc[df['trip_type'] == 'Docked-to-Docked','end_station_longitude'] = df['end_Station Longitude']

df.loc[df['trip_type'] == 'Docked-to-Dockless','start_station_latitude'] = df['start_Station Latitude']
df.loc[df['trip_type'] == 'Docked-to-Dockless','start_station_longitude'] = df['start_Station Longitude']

df.loc[df['trip_type'] == 'Dockless-to-Docked','end_station_latitude'] = df['end_Station Latitude']
df.loc[df['trip_type'] == 'Dockless-to-Docked','end_station_longitude'] = df['end_Station Longitude']

columns = ['start_Station Latitude','start_Station Longitude','end_Station Latitude','end_Station Longitude',
           'start_Station ID','start_Station ID DOMO','end_Station ID','end_Station ID DOMO','start_Station Name','end_Station Name','geometry']
df.drop(columns, inplace=True, axis=1)

df['start_station_name'] = df.start_station_name.fillna('Dockless Stationnement')
df['end_station_name'] = df.end_station_name.fillna('Dockless Stationnement')
df['route_stations'] = "From " + df['start_station_name'] + " To " + df['end_station_name']

In [12]:
display(df.shape)
display(df.dtypes)
df.head()

(4767654, 38)

rideable_type                      object
start_time                 datetime64[ns]
end_time                   datetime64[ns]
start_station_name                 object
start_station_id                   object
end_station_name                   object
end_station_id                     object
start_station_latitude            float64
start_station_longitude           float64
end_station_latitude              float64
end_station_longitude             float64
duration_sec                      float64
duration_min                      float64
trip_type                          object
user_type                          object
start_neigh                        object
end_neigh                          object
est_cost                          float64
is_equity                          object
bike_id                            object
rental_access_method               object
bike_share_for_all_trip            object
start_time_day                     object
start_time_month                  

,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,duration_sec,duration_min,trip_type,user_type,start_neigh,end_neigh,est_cost,is_equity,bike_id,rental_access_method,bike_share_for_all_trip,start_time_day,start_time_month,start_time_hour,start_time_year,end_time_day,end_time_month,end_time_hour,end_time_year,Quarter,route_neigh,eucl_dist_m,start_Has Kiosk,start_Dock Count,end_Has Kiosk,end_Dock Count,route_stations
0,docked_bike,2019-01-01 00:08:39.659,2019-01-01 00:38:06.848,Market St at Steuart St,16,Jackson Playground,115,37.794525,-122.394880,37.764965,-122.399025,1767.0,29.0,Docked-to-Docked,Customer,Financial District,Potrero Hill,2.0,,1705,,No,Tuesday,January,0,2019,Tuesday,January,1,2019,2019Q1,From Financial District To Potrero Hill,3252.0,True,25.0,True,27.0,From Market St at Steuart St To Jackson Playgr...
1,docked_bike,2019-01-01 00:14:55.921,2019-01-01 00:25:02.215,The Embarcadero at Steuart St,23,Berry St at King St,91,37.791401,-122.391038,37.771762,-122.398438,606.0,10.0,Docked-to-Docked,Subscriber,Financial District,Mission Bay,0.0,,5112,,No,Tuesday,January,0,2019,Tuesday,January,0,2019,2019Q1,From Financial District To Mission Bay,2281.0,True,23.0,True,23.0,From The Embarcadero at Steuart St To Berry St...
2,docked_bike,2019-01-01 00:15:11.324,2019-01-01 00:47:09.221,Montgomery St BART Station (Market St at 2nd St),21,Civic Center/UN Plaza BART Station (Market St ...,44,37.789625,-122.400811,37.781074,-122.411738,1917.0,31.0,Docked-to-Docked,Subscriber,Financial District,South of Market,0.0,,4838,,No,Tuesday,January,0,2019,Tuesday,January,1,2019,2019Q1,From Financial District To South of Market,1351.0,True,39.0,True,31.0,From Montgomery St BART Station (Market St at ...
3,docked_bike,2019-01-01 00:15:26.529,2019-01-01 00:47:19.626,Montgomery St BART Station (Market St at 2nd St),21,Civic Center/UN Plaza BART Station (Market St ...,44,37.789625,-122.400811,37.781074,-122.411738,1913.0,31.0,Docked-to-Docked,Subscriber,Financial District,South of Market,0.0,,5423,,No,Tuesday,January,0,2019,Tuesday,January,1,2019,2019Q1,From Financial District To South of Market,1351.0,True,39.0,True,31.0,From Montgomery St BART Station (Market St at ...
4,docked_bike,2019-01-01 00:16:36.845,2019-01-01 00:23:07.253,17th St at Valencia St,109,Valencia St at Cesar Chavez St,141,37.763333,-122.422055,37.747998,-122.420219,390.0,6.0,Docked-to-Docked,Subscriber,Mission Dolores,Bernal Heights,0.0,,5059,,No,Tuesday,January,0,2019,Tuesday,January,0,2019,2019Q1,From Mission Dolores To Bernal Heights,1706.0,True,23.0,True,19.0,From 17th St at Valencia St To Valencia St at ...


In [13]:
from google.colab import drive
drive.mount('/drive')

df.to_csv('/drive/MyDrive/Gcolab/modifiedData/lyft/masterdataset_lyft.csv',index=False)

Mounted at /drive


##### Feature Engineering, Round 3: Geospatial Attributes

---
###### Intersecting GEOJSON with the Master's dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# df = pd.read_csv(r'drive/MyDrive/Gcolab/modifiedData/lyft/masterdataset_lyft.csv',float_precision=None)
# display(df.shape)

In [ ]:
# fname = 'drive/MyDrive/Gcolab/rawdata/DataSF/elevation.geojson' # Your filepath here
# poly = gpd.read_file(fname)